In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Introducción

---

En esta notebook vamos a trabajar con el siguiente dataset:

https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

El objetivo de este analisis será predecir si lloverá o no al día siguiente.



## Ejercicio 1. EDA y Preprocesamiento

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.tree import DecisionTreeClassifier

import sklearn.metrics as metrics
from sklearn.model_selection import learning_curve



1. Leemos los datos del archivo weatherAUS.csv.zip

Contamos cuántos valores no-nulos hay

2. Eliminar las columnas que no nos interesan, entre ellas las que tienen pocos datos (menos de cien mil). Además, eliminar 'Location' y 'Date', ya que no nos interesa el lugar ni fecha (al menos en este análisis).

3. Eliminar todas las filas que tengan valores nulos.

4. Para simplificar el preprocesamiento, también eliminar todas las columnas que tengan valores categóricos. ¿Por qué no nos molesta eliminar `RainToday`?

5. Realizar un countplot para ver cuántos casos hay de lluvia y no-lluvia.

https://seaborn.pydata.org/generated/seaborn.countplot.html

Y hacer el `pairplot` para ver cómo se relacionan las variables. Recuerden que este gráfico puede llevar bastante tiempo. También recuerden que pueden agrandar el gráfico haciendo doble click en él.

Ayuda: usar como datos la muestra dada por `data.sample(frac = 0.1)`

¿Existe alguna feature, o par de features, que por sí solos sean buenos para predecir si lloverá o no al día siguiente?

Hay algunas que parecen *correlacionadas*. Tratamos de cuantificarlo.

In [ ]:
###############
# descomentar #
###############


# corr = data.drop(columns = ['RainTomorrow']).corr(method='pearson') # .corr is used for find corelation
# plt.figure(figsize=(14,14))
# sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
#            xticklabels= data.drop(columns = ['RainTomorrow']).columns, 
#            yticklabels= data.drop(columns = ['RainTomorrow']).columns,
#            cmap= 'coolwarm')


En base a la correlación, podemos descartar (o no) algunas variables. **Para pensar**, ¿por qué haríamos (o no) esto?

6. Convertir `RainTomorrow` a una variable númerica 1/0

### Datos de entrenamiento y casos *benchmark*

Generamos casos base contra los cuales comparar nuestros resultados.

1. Elegir variables de entrenamiento (empezar con dos) y construir la matriz de features X y el vector target y

2. Generar un modelo que diga siempre que NO va a llover y medir su exactitud (accuracy).

Y generar otro modelo que diga siempre que va a llover y medir su exactitud (accuracy).

## Ejercicio 2. Bagging

Separamos entre train y test

Recuerden que el objetivo de bagging es entrenar distintos modelos, donde cada uno vea distintas porciones del set de entrenamiento. Entonces, vamos a entrenar distintos árboles de decisión y mostrarles distintas porciones del set de datos. Lo vamos a hacer en un `for`.

1. Crear una lista vacía donde guardaremos los modelos entrenados y elegir cuántos modelos entrenar (Empezar por algún valor entre 5 y 10).

2. Entrenar cada modelo y guardar cada modelo entrenado en una lista. Para hacer el split, usar la función `train_test_split`. ¿Sobre qué conjunto van a hacer el split?¿Hay que fijar el `random_state`? 

3. Evaluar el accuracy de cada modelo usando el conjunto de test

Parecen estar un poco overfitteados, que era lo que esperábamos.

4. Evaluar el accuracy de todo el ensamble usando el conjunto de test. Vamos a hacerlo usando un promedio de las probabilidades que devuelven cada árbol. Si la probabilidad promedio es mayor a 0.5, clasificamos como positivo. Para ello:
    1. Inicializar un arreglo de probabilidades del tamaño de la cantidad de instancias del conjunto de test en ceros.

B. Recorrer la lista de modelos y predecir las probabilidades. Mirar como es el `shape` de ese arreglo predicho. Elegir las probabilidades que correspondan a la clase positiva. Luego, sumarlas al vector que definieron antes.

C. Dividir `probs_test_pred` por la cantidad de modelos

D. Crear las clases predichas (0s y 1s) a partir de comparar la probabilidad predicha con la probabilidad umbral (0.5).

Y evaluar la exactitud de todo el ensamble

5. Explorar el `BagginnClassfier` de scikit-learn y algunas de sus características. Usarlo para predecir sobre el train y test, y medir su desempeño.

6. Si usaron dos features, pueden graficar las fronteras de decisión

In [ ]:
###############
# descomentar #
###############


# N = 20 #para no graficar todos los puntos y saturar el grafico

# plt.figure(figsize = (8,6))

#Grafico Clasificador Sesgado
# ax = sns.scatterplot(x = X_test[::N].MaxTemp, y = X_test[::N].Humidity3pm, hue=y_test[::N], palette='Set2')
# xlim = ax.get_xlim()
# ylim = ax.get_ylim()
# xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
#                       np.linspace(*ylim, num=200))
# Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
# contours = ax.contourf(xx, yy, Z, alpha=0.3, cmap = 'Set2')


## Ejercicio 3. Random Forest

Random Forest, además de aplicar Bagging, también selecciona features al azar, de esa manera descorrelaciona aún más los distintos modelos de árbol creados.

1. Importar de scikit-learn el modelo `RandomForestClassifier`

2. Investigar sus parámetros. En particular, `n_estimators`, `max_features` y `oob_score`. Luego, crear y entrenar un modelo en el conjunto de train.

3. Evaluar su desempeño en el conjunto de train y de test.

4. ¿Cuál es su `oob_score_`?¿Y que son `feature_importances_`?

5. ¿Qué hay en la propiedad `estimators_`?

6. Elegir uno de los `estimators` y evaluar su desempeño sobre train y test.

¿Está overfiteado?¿Por qué la accuracy sobre el conjunto de train no es 1?

7. Hacer y graficar la curva de validación/complejidad para un modelo Random Forest en función del número de estimadores. No usamos CV porque puede llevar bastante tiempo. Si quieren, lo pueden probar después. Además, obtener su oob_score para graficar en la curva de complejidad (No se preocupen por los mensajes de warning).

In [ ]:
###############
# descomentar #
###############

# import warnings
# warnings.filterwarnings('ignore')

# train_accuracy = []
# test_accuracy = []
# oob_scores = []

# N_estimadores = [1,2,3,4,5,10,25,50,100,250,500,1000]
# for estimadores in N_estimadores:
#     print(estimadores)
#     clf = RandomForestClassifier(n_estimators=estimadores, n_jobs=-1, oob_score= True, random_state = 42)
#     clf.fit(X_train,y_train)
    
#     y_train_pred = clf.predict(X_train)
#     y_test_pred = clf.predict(X_test)
    
#     train_accuracy.append(metrics.accuracy_score(y_train, y_train_pred))
#     test_accuracy.append(metrics.accuracy_score(y_test, y_test_pred))
#     oob_scores.append(clf.oob_score_)
    
# train_accuracy = np.array(train_accuracy)
# test_accuracy = np.array(test_accuracy)
# oob_scores = np.array(oob_scores)

In [ ]:
###############
# descomentar #
###############

# plt.figure(figsize = (8,6))
# plt.plot(N_estimadores, train_accuracy, label = 'Train')
# plt.plot(N_estimadores, test_accuracy, label = 'Test')
# plt.plot(N_estimadores, oob_scores, label = 'OOB')
# plt.xlabel('Numero de estimadores')
# plt.ylabel('Accuracy')
# plt.legend()



8. Hacer y graficar la curva de aprendizaje (¿Qué es eso?) para un modelo con 250 estimadores. Puede llevar bastante tiempo, no se preocupen.

In [ ]:
# completar:
# clf = ...

###############
# descomentar #
###############

#train_sizes, train_scores, valid_scores = learning_curve(clf, X_train, y_train, 
#                                                         train_sizes = np.linspace(0.0001,1,10),
#                                                         scoring = 'accuracy', cv=5)

In [ ]:

###############
# descomentar #
###############

#plt.figure(figsize = (8,6))
#plt.plot(train_sizes, train_scores.mean(axis = 1), color = 'r')
#plt.plot(train_sizes, valid_scores.mean(axis = 1), color = 'g')

#plt.fill_between(train_sizes, train_scores.mean(axis = 1)- train_scores.std(axis = 1),
#                     train_scores.mean(axis = 1)+ train_scores.std(axis = 1), alpha=0.25,
#                     color="r")
#plt.fill_between(train_sizes, valid_scores.mean(axis = 1) - valid_scores.std(axis = 1),
#                     valid_scores.mean(axis = 1) + valid_scores.std(axis = 1), alpha=0.25, color="g")

#plt.ylim(0.5,1.1)


9. Si usaron dos features, pueden graficar las fronteras de decisión.

In [ ]:
###############
# descomentar #
###############

# N = 20 #para no graficar todos los puntos y saturar el grafico
# clf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)

# plt.figure(figsize = (8,6))

#Grafico Clasificador Sesgado
# ax = sns.scatterplot(x = X_test[::N].MaxTemp, y = X_test[::N].Humidity3pm, hue=y_test[::N], palette='Set2')
# xlim = ax.get_xlim()
# ylim = ax.get_ylim()
# xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
#                      np.linspace(*ylim, num=200))
# Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
# contours = ax.contourf(xx, yy, Z, alpha=0.3, cmap = 'Set2')


**Ejercicio**: elegir más features y volver a entrenar.

**Para pensar**: ¿qué otras métricas utilizarían para evaluar estos modelos, dadas las características particulares del problema? Comparar con los casos *benchmark* que hicieron.